In [ ]:
import numpy as np
from lmfit import Model, fit_report, minimize

print('Matlab ref omega: ', 1.752690409810814)

# Define the function to be fitted
def func(x, a, b, omega, phi, c):
    return a * np.exp(-b * x) * np.sin(omega * x + phi) + c

# Define the residual function
def residual(params, x, y):
    return y - func(x, **params)

# Load data from text files
xdata = np.loadtxt('xdata.txt')
ydata = np.loadtxt('data150_6.txt')

# Mask 20% of the data
n_mask = int(len(ydata) * 0.0)
mask_indices = np.random.choice(len(ydata), size=n_mask, replace=False)
ydata[mask_indices] = np.nan
# Define the boolean mask
mask = np.logical_not(np.isnan(ydata))
# Apply the mask to the data
xdata = xdata[mask]
ydata = ydata[mask]


# Define the model to be fitted
model = Model(func)

# Set initial parameter values and bounds
params = model.make_params(a=1.0, b=1.0, omega=1.0, phi=1.0, c=1.0)


params['a'].min = 0.0
params['b'].min = 0.0
params['omega'].min = 0.0
params['phi'].min = -np.pi
params['phi'].max = np.pi
params['a'].max = 5.0
params['b'].max = 5.0
params['omega'].max = 5.0
params['c'].min=0.0
params['c'].max=5.0



# Fit the model to the data using different methods
result_leastsq = model.fit(ydata, params, x=xdata, method='leastsq')
result_least_squares_bounds = model.fit(ydata, params, x=xdata, method='least_squares')
result_nelder = minimize(residual, params, args=(xdata, ydata), method='nelder')
result_bfgs = minimize(residual, params, args=(xdata, ydata), method='bfgs')
result_differential_evolution = minimize(residual, params, args=(xdata, ydata), method= 'differential_evolution')
result_basinhopping = minimize(residual, params, args=(xdata, ydata), method= 'basinhopping')
#result_brute = minimize(residual, params, args=(xdata, ydata), method= 'brute')


# Print the fit results for each method
print("Results using least sq:")
print(fit_report(result_leastsq))
print("\nResults using least squares with bounds:")
print(fit_report(result_least_squares_bounds))
print("\nResults using Nelder-Mead optimization:")
print(fit_report(result_nelder))
print("\nResults using BFGS optimization:")
print(fit_report(result_bfgs))
print("\nResults using differential_evolution optimization:")
print(fit_report(result_differential_evolution))

print("\nResults using basinhopping optimization:")
print(fit_report(result_basinhopping))

# print("\nResults using brute optimization:")
# print(fit_report(result_brute))

# Create a list of dictionaries to store the results
results = []

# Add the results from each method to the list
results.append({
    'method': 'leastsq',
    'b': result_leastsq.params['b'].value,
    'omega': result_leastsq.params['omega'].value
})
results.append({
    'method': 'least_squares',
    'b': result_least_squares_bounds.params['b'].value,
    'omega': result_least_squares_bounds.params['omega'].value
})
results.append({
    'method': 'nelder',
    'b': result_nelder.params['b'].value,
    'omega': result_nelder.params['omega'].value
})
results.append({
    'method': 'bfgs',
    'b': result_bfgs.params['b'].value,
    'omega': result_bfgs.params['omega'].value
})
results.append({
    'method': 'differential_evolution',
    'b': result_differential_evolution.params['b'].value,
    'omega': result_differential_evolution.params['omega'].value
})

results.append({
    'method': 'basinhopping',
    'b': result_basinhopping.params['b'].value,
    'omega': result_basinhopping.params['omega'].value
})
# results.append({
#     'method': 'brute',
#     'b': result_brute.params['b'].value,
#     'omega': result_brute.params['omega'].value
# })

# Write the results to a CSV file
import csv

with open('results_{0}.csv'.format(n_mask), 'w', newline='') as csvfile:
    fieldnames = ['method', 'b', 'omega']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for result in results:
        writer.writerow(result)

